In [151]:
import requests
import config
from bs4 import BeautifulSoup as BS
import mysql.connector
import pandas as pd

In [125]:
cnx = mysql.connector.connect(
  host=config.hostj,
  user=config.userj,
  passwd=config.pwj,
  database = 'nfl_draft_info')

cursor = cnx.cursor()

In [136]:
cnx.commit()

In [128]:
cursor.execute("""SHOW TABLES""")
cursor.fetchall()

[('draft_picks',), ('nfl_combine_info',)]

In [161]:
cursor.execute("""SELECT * FROM draft_picks WHERE name LIKE '%Url%'""")
cursor.fetchall()

[(578, 71, 'Nate Burleson', 'WR', 'Nevada', 2003),
 (655, 148, 'Dan Curley', 'FB', 'East. Washington', 2003),
 (3579, 10, 'Todd Gurley', 'RB', 'Georgia', 2015),
 (4850, 9, "Brian Urlacher', ' ', 'HOF", 'LB', 'New Mexico', 2000)]

In [127]:
cursor.execute("""CREATE TABLE draft_picks(
                id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
                pick_number INT,
                name VARCHAR(100),
                position VARCHAR(10),
                college VARCHAR(50),
                year INT
                )""")

In [74]:
def get_draft_info(year):
    draft_page = requests.get("https://www.pro-football-reference.com/years/" + str(year) + "/draft.htm")
    draft_soup = BS(draft_page.content, 'html.parser')
    draft_html = list(draft_soup.children)[3]
    draft_body = list(draft_html.children)[3]
    draft_table = list(draft_body.children)[1]
    draft_info = list(draft_table.children)[11]
    draft_info_table = list(draft_info.children)[9]
    draft_detail = list(draft_info_table.children)[3]
    dtable = draft_detail.find("table")
    drows = dtable.findAll('tr')
    ddata = [[td.findChildren(text=True) for td in tr.findAll("td")] for tr in drows]
    ddata2 = []
    for i in ddata:
        if len(i) == 28:
            ddata2.append({'pick': i[0], 'name': i[2], 'position': i[3], 'college': i[-2], 'year': year})
    return ddata2

In [75]:
results00 = get_draft_info(2000)

In [76]:
results00

[{'pick': ['1'],
  'name': ['Courtney Brown'],
  'position': ['DE'],
  'college': ['Penn St.'],
  'year': 2000},
 {'pick': ['2'],
  'name': ['LaVar Arrington'],
  'position': ['LB'],
  'college': ['Penn St.'],
  'year': 2000},
 {'pick': ['3'],
  'name': ['Chris Samuels'],
  'position': ['T'],
  'college': ['Alabama'],
  'year': 2000},
 {'pick': ['4'],
  'name': ['Peter Warrick'],
  'position': ['WR'],
  'college': ['Florida St.'],
  'year': 2000},
 {'pick': ['5'],
  'name': ['Jamal Lewis'],
  'position': ['RB'],
  'college': ['Tennessee'],
  'year': 2000},
 {'pick': ['6'],
  'name': ['Corey Simon'],
  'position': ['DT'],
  'college': ['Florida St.'],
  'year': 2000},
 {'pick': ['7'],
  'name': ['Thomas Jones'],
  'position': ['RB'],
  'college': ['Virginia'],
  'year': 2000},
 {'pick': ['8'],
  'name': ['Plaxico Burress'],
  'position': ['WR'],
  'college': ['Michigan St.'],
  'year': 2000},
 {'pick': ['9'],
  'name': ['Brian Urlacher', ' ', 'HOF'],
  'position': ['LB'],
  'college': [

In [77]:
for sub in results00:
    for key in sub:
        sub[key] = str(sub[key]).strip("['']")

In [78]:
for sub in results00:
    sub['pick'] = int(sub['pick'])

In [79]:
for sub in results00:
    sub['year'] = int(sub['year'])

In [88]:
results00[0]

{'pick': 1,
 'name': 'Courtney Brown',
 'position': 'DE',
 'college': 'Penn St.',
 'year': 2000}

In [91]:
draft_list = []
for player in results00:
    draft_list.append((player['pick'], player['name'], player['position'], player['college'], player['year']))

In [98]:
stmt = "INSERT INTO nfl_draft_info.draft_picks (pick_number, name, position, college, year) VALUES (%s, %s, %s, %s, %s)"
cursor.executemany(stmt, draft_list)

In [105]:
def insert_into_draft_table(year1, year2):
    for i in range(year1, year2):
        draft_results = get_draft_info(i)
        for sub in draft_results:
            for key in sub:
                sub[key] = str(sub[key]).strip("['']")
        for sub in draft_results:
            sub['pick'] = int(sub['pick'])
        for sub in draft_results:
            sub['year'] = int(sub['year'])
        draft_list = []
        for player in draft_results:
            draft_list.append((player['pick'], player['name'], player['position'], player['college'], player['year']))
        stmt = "INSERT INTO nfl_draft_info.draft_picks (pick_number, name, position, college, year) VALUES (%s, %s, %s, %s, %s)"
        cursor.executemany(stmt, draft_results)
        cnx.commit()    
    

In [111]:
def get_draft_list(year1, year2):
    draft_results_list = []
    for i in range(year1, year2):
        draft_results = get_draft_info(i)
        for sub in draft_results:
            for key in sub:
                sub[key] = str(sub[key]).strip("['']")
        for sub in draft_results:
            sub['pick'] = int(sub['pick'])
        for sub in draft_results:
            sub['year'] = int(sub['year'])
        draft_results_list.append(draft_results)
    return draft_results_list
        #draft_list = []
        #for player in draft_results:
            #draft_list.append((player['pick'], player['name'], player['position'], player['college'], player['year']))
    #return draft_list

In [119]:
def get_full_player_list(year1, year2):
    draft_results_ld = get_draft_list(year1, year2)
    draft_list = []
    for year in draft_results_ld:
        for player in year:
            draft_list.append((player['pick'], player['name'], player['position'], player['college'], player['year']))
    return draft_list
#             stmt = "INSERT INTO nfl_draft_info.draft_picks (pick_number, name, position, college, year) VALUES (%s, %s, %s, %s, %s)"
#             cursor.executemany(stmt, draft_results_ld)
#             cnx.commit()

In [159]:
draft_list = get_full_player_list(2000,2001)

In [122]:
def insert_into_sql(dlist):
    stmt = "INSERT INTO nfl_draft_info.draft_picks (pick_number, name, position, college, year) VALUES (%s, %s, %s, %s, %s)"
    cursor.executemany(stmt, dlist)
    cnx.commit()

In [160]:
insert_into_sql(draft_list)

In [163]:
cursor.execute("""SELECT id, pick_number, name, position, year FROM draft_picks""")
draft_picks = cursor.fetchall()

In [164]:
draft_picks[0]

(1, 1, 'Michael Vick', 'QB', 2001)

In [165]:
df = pd.DataFrame(draft_picks, columns = ['id', 'pick', 'name', 'position', 'year'])
df = df.set_index('id')
df.tail()

,pick,name,position,year
id,,,,
5091,250,Ethan Howell,WR,2000
5092,251,DaShon Polk,LB,2000
5093,252,Rondell Mealey,RB,2000
5094,253,Alfonso Boone,DT,2000
5095,254,Michael Green,DB,2000


In [174]:
df[df['name'].str.contains('HOF')]

,pick,name,position,year
id,,,,
1313,34,"""D'Qwell Jackson""",LB,2006


In [171]:
df = df.replace("Brian Urlacher', ' ', 'HOF", "Brian Urlacher")

In [172]:
df.head()

,pick,name,position,year
id,,,,
1,1,Michael Vick,QB,2001
2,2,Leonard Davis,T,2001
3,3,Gerard Warren,DT,2001
4,4,Justin Smith,DE,2001
5,5,Ladanian Tomlinson,RB,2001
